<a href="https://colab.research.google.com/github/Artemiy2708/ML/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22%F0%9F%90%AFH_AA_Astashkin_ipynb%22%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<small><font color=gray>Notebook author: <a href="https://www.linkedin.com/in/olegmelnikov/" target="_blank">Oleg Melnikov</a>, ©<a href="https://apps.ep.jhu.edu/course-homepages/3765-605-742-deep-neural-networks" target="_blank">JHU</a> 2021 onwards</font></small><hr style="margin:0;background-color:silver">

# **[2🏆🌌Stellar](https://www.kaggle.com/c/jan-30-22stellar/rules)**

See [**instructions**](https://colab.research.google.com/drive/1riOGrE_Fv-yfIbM5V4pgJx4DWcd92cZr#scrollTo=ITaPDPIQEgXV) for running and naming your Colab notebooks.



In [ ]:
from google.colab import drive; drive.mount('/content/drive')   # OK to enable, if your kaggle.json is stored in Google Drive

Mounted at /content/drive


In [ ]:
!pip -q install --upgrade --force-reinstall --no-deps kaggle > log  # upgrade kaggle package (to avoid a warning)
!mkdir -p ~/.kaggle                                           # .kaggle folder must contain kaggle.json for kaggle executable to properly authenticate you to Kaggle.com
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json >log  # First, download kaggle.json from kaggle.com (in Account page) and place it in the root of mounted Google Drive
#!cp kaggle.json ~/.kaggle/kaggle.json > log                   # Alternative location of kaggle.json (without a connection to Google Drive)
!chmod 600 ~/.kaggle/kaggle.json                              # give only the owner full read/write access to kaggle.json
!kaggle config set -n competition -v jan-30-22stellar          # set the competition context for the next few kaggle API calls. !kaggle config view - shows current settings
!kaggle competitions download >> log                          # download competition dataset as a zip file
!unzip -o *.zip >> log                                        # Kaggle dataset is copied as a single file and needs to be unzipped.
lb =!kaggle competitions leaderboard --show                        # print public leaderboard

- competition is now set to: jan-30-22stellar
100% 7.58M/7.58M [00:00<00:00, 259MB/s]


In [ ]:
%%time
%%capture
%reset -f
from IPython.core.interactiveshell import InteractiveShell as IS; IS.ast_node_interactivity = "all" 
import numpy as np, pandas as pd, time, matplotlib.pyplot as plt, seaborn as sns, os, tqdm, re, sys, cv2, skimage
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression as LR
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA, LinearDiscriminantAnalysis as LDA
ToCSV = lambda df, fname: df.round(2).to_csv(f'{fname}.csv', index_label='id') # rounds values to 2 decimals

class Timer():
  def __init__(self, lim:'RunTimeLimit'=60): self.t0, self.lim, _ = time.time(), lim, print(f'⏳ started. You have {lim} sec. Good luck!')
  def ShowTime(self):
    msg = f'Runtime is {time.time()-self.t0:.0f} sec'
    print(f'\033[91m\033[1m' + msg + f' > {self.lim} sec limit!!!\033[0m' if (time.time()-self.t0-1) > self.lim else msg)

np.set_printoptions(linewidth=100, precision=2, edgeitems=2, suppress=True)
pd.set_option('max_columns', 20, 'precision', 2, 'display.max_rows', 4)

CPU times: user 1.03 s, sys: 144 ms, total: 1.18 s
Wall time: 2.17 s


In [ ]:
df = pd.read_csv('XY_Stellar.csv'); df

,alpha,delta,u,g,r,i,z,run_ID,cam_col,field_ID,redshift,plate,MJD,fiber_ID,Class
0,11.64,21.28,26.28,26.15,24.05,18.87,19.00,8848,5,272,0.84,7740,56824,833,NaN
1,173.09,42.21,22.51,22.83,22.21,19.55,19.96,4156,3,486,0.81,9041,58067,428,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199998,131.31,44.27,24.07,24.64,21.63,19.20,19.03,7076,3,251,0.55,6014,56166,1021,G
199999,22.59,0.25,25.30,25.56,24.09,19.41,19.96,5164,4,511,1.26,9590,57969,878,G


In [ ]:
df.info()   # observe datatypes and any missing values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 15 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   alpha     200000 non-null  float64
 1   delta     200000 non-null  float64
 2   u         200000 non-null  float64
 3   g         200000 non-null  float64
 4   r         200000 non-null  float64
 5   i         200000 non-null  float64
 6   z         200000 non-null  float64
 7   run_ID    200000 non-null  int64  
 8   cam_col   200000 non-null  int64  
 9   field_ID  200000 non-null  int64  
 10  redshift  200000 non-null  float64
 11  plate     200000 non-null  int64  
 12  MJD       200000 non-null  int64  
 13  fiber_ID  200000 non-null  int64  
 14  Class     160000 non-null  object 
dtypes: float64(8), int64(6), object(1)
memory usage: 22.9+ MB


In [ ]:
# Change string labels to numbers in order of increasing size of the entity (Star < Quasi Star < Galaxy)
# df.Class = df.Class.apply(lambda C: -1 if C=='S' else 0 if C=='Q' else 1 if C=='G' else None) 

In [ ]:
vX = df.query('Class!=Class').drop('Class', axis=1)  # slice a test sample
tXY = df.query('Class==Class')                       # slice training sample
tX, tY = tXY.drop('Class', axis=1), tXY.Class        # split into training I/O

In [ ]:
def ScatterCorrHist(df):
  def corrdot(*args, **kwargs):
    # credit: https://stackoverflow.com/questions/48139899
    corr_r = args[0].corr(args[1], 'pearson')
    corr_text = f"{corr_r:2.2f}".replace("0.", ".")
    ax = plt.gca();
    ax.set_axis_off();
    msz = abs(corr_r) * 5000   # marker size
    fsz = abs(corr_r) * 40 + 5 # font size
    ax.scatter([.5], [.5], msz, [corr_r], alpha=0.5, cmap='coolwarm', vmin=-1, vmax=1, transform=ax.transAxes)
    ax.annotate(corr_text, [.5, .5,],  xycoords="axes fraction", ha='center', va='center', fontsize=fsz)

  sns.set(style='white', font_scale=.8);
  g = sns.PairGrid(df, aspect=1, diag_sharey=False);
  g.fig.set_size_inches(20,10)
  g.map_lower(sns.regplot, lowess=True, ci=False, line_kws={'color':'red'}, scatter_kws={'s':1});
  g.map_diag(sns.histplot, kde_kws={'color':'black'});
  g.map_upper(corrdot);
  g.fig.suptitle("Scatter plot, Correlations and histograms on diagonal", y=1);
  _ = plt.subplots_adjust(hspace=0.02, wspace=0.02);
  _ = plt.show();

# ScatterCorrHist(tXY.head(200))

In [ ]:
tmr = Timer()

⏳ started. You have 60 sec. Good luck!


<hr color=red>

<font size=5>⏳</font> <strong><font color=orange size=5>Your Code, Documentation, Ideas and Timer - All Start Here...</font></strong>

**Student's Section** (between ⏳ symbols): add your code and documentation here.

## **Task 1. Preprocessing Pipeline**
 
Explain elements of your preprocessing pipeline i.e. feature engineering, subsampling, clustering, dimensionality reduction, etc. 
1. Why did you choose these elements? (Something in EDA, prior experience,...? Btw, EDA is not required)
1. How do you evaluate the effectiveness of these elements? 
1. What else have you tried that worked or didn't? 

**Student's answer:**
we made outliers clearance as usual using IQR, before checking that we don't have null values in our dataframe. See some graphs , scatterplots noted that we have categorial ordinal variable hided in int, some distributions similar to lognormal, but in modelling approoach I explained what did'n work. I trie to get away from collinearity buy droping rows, and it worked in some , looking in the collinear matrices, however they still have collinear issuues, and I left only 3 features. 

## **Task 2. Modeling Approach**
Explain your modeling approach, i.e. ideas you tried and why you thought they would be helpful. 

1. How did these decisions guide you in modeling?
1. How do you evaluate the effectiveness of these elements? 
1. What else have you tried that worked or didn't? 

**Student's answer:**
qda, lda, and logreg models applied. we were droppping rows, using permutations and seeing which score gives higher ot train test data with labels. Some scaling didn't help, np.log to features with lognormal distributions decreased our score. After finding optimal columns we leave, we try now to drop find optimal hyperparapetrs using gridsearchcv, but is working very long, so don't apply it to all parametres, only to reg_param of qda- 0.1. If we aplly more np.log to lognormal distributions , it should work better than it seems to be due to theory from ISLR, but somehow when we normalize some data we get not the result we wanr ans score decreases. Moreover , i think we did not use ordinality of response variable, i think it should also increase the score a litlle bit, but kaggle stopped working and requires telephone number, so i don't want my work to be nullified for different leaderboard and score here, so i just leave these code commented, but kepp in mind that should increase our score. We did not try to use logreg as ,reading the islr it stated that with gaussian distribution, and here many features are gaussian or can  be trasfered to it it is better to apply lda or qda.

Below is a baseline model that produces the result on Kaggle leaderboard (LB).

In [ ]:
df_predict=df[df['Class'].isna()]
df_new=df[df['Class'].notnull()]
df_new.isna().sum().sum()

0

In [ ]:
q1_alpha=df_new['alpha'].quantile(0.25)-1.5*(df_new['alpha'].quantile(0.75)-df_new['alpha'].quantile(0.25))
q3_alpha=df_new['alpha'].quantile(0.75)+1.5*(df_new['alpha'].quantile(0.75)-df_new['alpha'].quantile(0.25))
df_new=df_new.loc[(df_new['alpha']>=q1_alpha) & (df_new['alpha']<=q3_alpha)]

q1_delta=df_new['delta'].quantile(0.25)-1.5*(df_new['delta'].quantile(0.75)-df_new['delta'].quantile(0.25))
q3_delta=df_new['delta'].quantile(0.75)+1.5*(df_new['delta'].quantile(0.75)-df_new['delta'].quantile(0.25))
df_new=df_new.loc[(df_new['delta']>=q1_delta) & (df_new['delta']<=q3_delta)]

q1_run_ID	=df_new['run_ID'].quantile(0.25)-1.5*(df_new['run_ID'].quantile(0.75)-df_new['run_ID'].quantile(0.25))
q3_run_ID	=df_new['run_ID'].quantile(0.75)+1.5*(df_new['run_ID'].quantile(0.75)-df_new['run_ID'].quantile(0.25))
df_new=df_new.loc[(df_new['run_ID']>=q1_run_ID) & (df_new['run_ID']<=q3_run_ID)]

q1_u=df_new['u'].quantile(0.25)-1.5*(df_new['u'].quantile(0.75)-df_new['u'].quantile(0.25))
q3_u=df_new['u'].quantile(0.75)+1.5*(df_new['u'].quantile(0.75)-df_new['u'].quantile(0.25))
df_new=df_new.loc[(df_new['u']>=q1_u) & (df_new['u']<=q3_u)]

q1_g=df_new['g'].quantile(0.25)-1.5*(df_new['g'].quantile(0.75)-df_new['g'].quantile(0.25))
q3_g=df_new['g'].quantile(0.75)+1.5*(df_new['g'].quantile(0.75)-df_new['g'].quantile(0.25))
df_new=df_new.loc[(df_new['g']>=q1_g) & (df_new['g']<=q3_g)]

q1_z=df_new['z'].quantile(0.25)-1.5*(df_new['z'].quantile(0.75)-df_new['z'].quantile(0.25))
q3_z=df_new['z'].quantile(0.75)+1.5*(df_new['z'].quantile(0.75)-df_new['z'].quantile(0.25))
df_new=df_new.loc[(df_new['z']>=q1_z) & (df_new['z']<=q3_z) ]

q1_i=df_new['i'].quantile(0.25)-1.5*(df_new['i'].quantile(0.75)-df_new['i'].quantile(0.25))
q3_i=df_new['i'].quantile(0.75)+1.5*(df_new['i'].quantile(0.75)-df_new['i'].quantile(0.25))
df_new=df_new.loc[(df_new['i']>=q1_i) & (df_new['i']<=q3_i) ]

q1_r=df_new['r'].quantile(0.25)-1.5*(df_new['r'].quantile(0.75)-df_new['r'].quantile(0.25))
q3_r=df_new['r'].quantile(0.75)+1.5*(df_new['r'].quantile(0.75)-df_new['r'].quantile(0.25))
df_new=df_new.loc[(df_new['r']>=q1_r) & (df_new['r']<=q3_r) ]

In [ ]:
import numpy as np, pandas as pd, time, matplotlib.pyplot as plt, seaborn as sns
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression as LR
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA, LinearDiscriminantAnalysis as LDA
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split as tts
#df_new.Class = df_new.Class.apply(lambda C: 1 if C=='S' else 2 if C=='Q' else 3 if C=='G' else None) 
#df_predict.Class= df_predict.Class.apply(lambda C: 1 if C=='S' else 2 if C=='Q' else 3 if C=='G' else None) 
#df_new.field_ID=np.log(df_new.field_ID)
#df_new.redshift=np.log(df_new.redshift)
X_train, X_test, y_train, y_test = tts(
    df_new.drop(['Class','run_ID', 'i', 'z', 'r', 'delta', 'field_ID', 'fiber_ID', 'plate'], axis=1), df_new["Class"], test_size=0.2,
    random_state=0
)
poly = PolynomialFeatures(degree=2)    # add non-linear features (powers and interactions)
X_train = poly.fit_transform(X_train.select_dtypes(include=np.number))  # create object on training set
X_test = poly.transform(X_test.select_dtypes(include=np.number))      # apply the same object to test set
vXp  = poly.transform(df_predict.drop(['Class','run_ID', 'delta', 'field_ID', 'fiber_ID', 'i', 'z', 'r', 'plate'], axis=1).select_dtypes(include=np.number))       # apply the same object to test set

m = QDA(reg_param=0.1)
# m = LR(C=.0001, max_iter=100, n_jobs=-1, multi_class='auto', random_state=0) # always seed your experiments for reproducibility
print(m.fit(X_train, y_train).score(X_test, y_test))   # fit the model and show test error

/usr/local/lib/python3.7/dist-packages/sklearn/discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


0.9558364747172754


In [ ]:
pY = pd.DataFrame(m.predict(vXp), index=range(1,len(df_predict)+1), columns=['Class'])  # ensure that labels and observations are in corresponding order
pY
ToCSV = lambda df, fname: df.round(2).to_csv(f'{fname}.csv', index_label='id')
ToCSV(pY, 'MySubmission')

,Class
1,G
2,Q
...,...
39999,G
40000,S


# **References:**

1. Remember to cite your sources here as well! At the least, your textbook should be cited. Google Scholar allows you to effortlessly copy/paste an APA citation format for books and publications. Also cite StackOverflow, package documentation, and other meaningful internet resources to help your peers learn from these (and to avoid plagiarism claims).

<font size=5>⌛</font> <strong><font color=orange size=5>Do not exceed competition's runtime limit!</font></strong>

<hr color=red>


In [ ]:
tmr.ShowTime()    # measure Colab's runtime. Do not remove. Keep as the last cell in your notebook.

Runtime is 1 sec


## 💡**Starter Ideas**

1. Tune model hyperparameters
1. Try to linear and non-linear feature normalization: shift/scale, log, divide features by features (investigate scatterplot matrix)
1. Try higher order feature interactions and polynomial features on a small subsample. Then identify key features or select key principal components. The final model can be trained on a larger or even full training sample. You can use [PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) to reduce the feature set
1. Do a thorough EDA: look for feature augmentations that result in linear decision boundaries between pairs of classes.
1. Evaluate predictions and focus on poorly predicted "groups":
  1. Strongest missclassifications. E.g. the model is very confident about the wrong label
  1. Evaluate predictions near decision boundaries.
1. Do scatter plots show piecewise linear shape? Can a separate linear model be used on each support, or can the pattern be linearized via transformations?
1. How are date/categorical features treated by the model? Is there a [better way](https://www.google.com/search?q=ways+to+encode+categorical+data) to encode these (perhaps, ordinal) features? 
  1. E.g. you could replace codes (or groups of codes) with their frequencies, which may capture the implied "distance" or rarity between category levels.
  1. If encoding ordinal features with integers, should non-equidistant values be considered?
1. Learn astronomy domain and features: [🎦](https://www.youtube.com/results?search_query=Quasi-star), [quasi-star](https://en.wikipedia.org/wiki/Quasi-star), [star](https://en.wikipedia.org/wiki/Star), [galaxy](https://en.wikipedia.org/wiki/Galaxy), [📃](https://arxiv.org/abs/2112.02026)
